In [2]:
!pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4 faiss-cpu

In [1]:
import getpass
import os
import requests
import json
import time

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings

from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool

import faiss
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from pydantic import BaseModel

from dotenv import load_dotenv
from sber_utils import get_access_token
from typing import Literal

load_dotenv('.env')
auth_token = get_access_token(os.environ.get("CLIENT_ID"),
                              os.environ.get("AUTH_KEY"))
auth_token

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/YEAR 2/INTRO_LLM/llm_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.LGNtNMBmWqfd5BwVy12FE12Rolst-9dANTNGLfvyaJmVMVO0sYXluhhJvz8c5fw1_qzEllTjy6aITPuo4R1xsB7eNwFG7vN7vssqJ32oMTCpcULXGNHZ24hk_2RsnHPSe9ojrQGj6pqrl9tBDBHEH9r94mdOC_meSBFMEPK2UgaYoc5OKGBndlGAIJMA45_ogDXgpoOMQHbcNpNhrcnRHhxoF8Bko89n0khnkE4KrFvJdfMdOUGKAag7CLCSjMntWx1dNCEx-s8iTLSCdSlhJft8kkO-M-ecpZH89ymtd7Ss4xL2QyCfdVNgPSpoGTz5vkWjl071vjWJ9D-Y4ajFbQ.oUe_7RCvpvOo8W5omGZUiQ.Xohag7r2jaB8vvRFXb6V7yO-74wTfiyjT1QziKow_8L6tmMNHwKLRyvCxQXy3Tquynf9V04YgG7kQRirP_Naid0_VhCdjB55yE0X9oDBlEiO0brfM2zBX_S4poyzyjuLLjrJE42DkL4oTSwIMFjg0Z_M-58BO4rmf-pIm_KvKm004o-rCBzsy5VoSoOBcKVbR1qyP2uwn4tOubcNAjXBd1i12vVsxfSsqwMmxVh12ybvw3CFNJVKPYrUqTjy-rR5X09ghUrrzpPClTWF9CXCuSs1a8TFrU28uWctwLGilx4RC2ZfY_nhPViGTGaXtpz4TnglArw1VXomu2s9BWxEM19Ch3DqATbWQemqV_AEsSFQMViFvn-kxzbxhZHMGa2cHSu5X8a9cjWSXkIRGL7b4zb_CL073-SI3SQVNcevmiheKQ1RcosuTicodG87WOY7j-dRc7rXhew8IcAbPsXrMcJcgzWqmRkz6vv4PYSxKZ8Usg68UE3SFHwDTnXz08B0HgwrDO1O1lDCE30tODLOPrWSlwRbqHjOYWRm-f5Pa0zgz

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [13]:
class SimpleChatBot:
    def __init__(self, llm: GigaChat, memory_type: Literal['simple', 'window', 'summary'] = 'simple',
                 **memory_kwargs: dict):
        if memory_type == 'simple':
            self.memory = ConversationBufferMemory()
        elif memory_type == 'window':
            self.memory = ConversationBufferWindowMemory(**memory_kwargs)
        elif memory_type == 'summary':
            self.memory = ConversationSummaryMemory(llm=llm, **memory_kwargs)
        else:
            raise ValueError('Wrong value for `memory_type` arg')
        
        self.conversation = ConversationChain(
            llm=llm,
            verbose=False,
            memory=self.memory
        )
 
    def _respond(self, user_input):
        return self.conversation.predict(input=user_input)
    
    def print_memory(self):
        print(self.memory.load_memory_variables({}))

    def run(self):
        while True:
            user_input = input('Введите сообщение:').strip()
            if not user_input:
                print("До свидания!")
                break
            response = self._respond(user_input)
            print(response)


In [19]:
giga = GigaChat(access_token=auth_token, model="GigaChat", timeout=30, verify_ssl_certs=False)

Now let's check how it works with each type of memory

In [17]:
# giga_key = os.environ.get("SB_AUTH_DATA")
# giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=5)
chat.run()
chat.print_memory()

Здравствуйте! Мне очень приятно общаться с вами. К сожалению, я не имею физического тела и не могу есть, поэтому у меня нет любимого блюда. Но если бы я мог, то выбрал бы что-то изысканное и необычное, например, трюфели или фуа-гра. А какие у вас любимые сладости?
До свидания!
{'history': 'Человек приветствует AI и интересуется его любимым блюдом. AI отвечает, что, поскольку у него нет физического тела, у него нет любимого блюда, но если бы он мог есть, то выбрал бы трюфели или фуа-гра. Затем человек делится своими любимыми сладостями.'}


In [ ]:
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

In [ ]:
chat = SimpleChatBot(giga, 'simple', memory_window_size=2)
chat.run()
chat.print_memory()

Please make a short report abount differences between used memory types

Report:

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [31]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

from langchain.tools import tool
from langchain.agents import AgentExecutor, create_gigachat_functions_agent
from langchain.chat_models.gigachat import GigaChat
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import Optional
import requests

openweather_key = os.environ.get("WEATHER_API_KEY")

few_shot_examples = [
    {
        "request": "Какая погода сейчас в Ставрополе?",
        "params": {"city_name": "Stavropol"},
    },
    {
        "request": "Подскажи, сколько градусов в Лондоне?",
        "params": {"city_name": "London"},
    },
]

class WeatherResult(BaseModel):
    weather: str = Field(description="Current weather", default='-')
    weather_description: str = Field(description="Detailed description of current weather", default='-')
    temp: float = Field(description="Temerature value", default=0)
    pressure: float = Field(description="Atmospheric pressure", default=0)
    humidity: float = Field(description="Humidity level", default=0)
    wind_speed: float = Field(description="Wind speed", default=0)
    city_name: str = Field(description="City name", default='-')
    country: str = Field(description="Country name", default='-')
    status: str = Field(description="Response status", default='-')


@tool(few_shot_examples=few_shot_examples)
def get_wheather(city_name: Optional[str] = Field(None, description="City name")) -> WeatherResult:
    """Get current weather forecast for given city"""
    params = {
        'q': city_name,
        'appid': os.environ.get("WEATHER_API_KEY"),
        'units': 'metric'
    }
    try:
        response = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params)
        response.raise_for_status()
        response = response.json()
        weather_full = ', '.join([res['main'] for res in response['weather']])
        weather_description_full = ', '.join([res['description'] for res in response['weather']])
        return WeatherResult(weather=weather_full,
                             weather_description=weather_description_full,
                             temp=response['main']['temp'],
                             pressure=response['main']['pressure'],
                             humidity=response['main']['humidity'],
                             wind_speed=response['wind']['speed'],
                             city_name=response['name'],
                             country=response['sys']['country'],
                             status='ok')
    except Exception:
        return WeatherResult(status="Error")




class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function
        self.agent_executor = AgentExecutor(
            agent=create_gigachat_functions_agent(self.llm, [agent_function]),
            tools=[agent_function],
            verbose=True,
        )

    def run(self, user_input: str):
        chat_history = [
            SystemMessage(content="You're a useful weather assistant. Follow the user's commands, tell about weather in city provided and give some advices.")
        ]
        try:
            result = self.agent_executor.invoke(
                {
                    "chat_history": chat_history,
                    "input": user_input,
                }
            )
            print(f"Bot: {result['output']}")
        except Exception as e:
            print(f"Error: {str(e)}")

Let's check it

In [32]:
auth_token = get_access_token(os.environ.get("CLIENT_ID"),
                              os.environ.get("AUTH_KEY"))
giga_pro = GigaChat(access_token=auth_token, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_wheather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/YEAR 2/INTRO_LLM/llm_venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `get_wheather` with `{'city_name': 'Ставрополь'}`


weather='Clouds' weather_description='scattered clouds' temp=2.16 pressure=1024.0 humidity=93.0 wind_speed=2.0 city_name='Stavropol' country='RU' status='ok'Сейчас в Ставрополе облачно, местами видны рассеянные облака. Температура воздуха составляет +2°C. Атмосферное давление - 1024 гПа, влажность воздуха высокая - 93%. Ветер умеренный, около 2 м/с.

> Finished chain.
Bot: Сейчас в Ставрополе облачно, местами видны рассеянные облака. Температура воздуха составляет +2°C. Атмосферное давление - 1024 гПа, влажность воздуха высокая - 93%. Ветер умеренный, около 2 м/с.


## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [23]:
from langchain.tools import Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser

openweather_key = os.environ.get("OPENWEATHER_API_KEY")

def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)


#TODO
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> str:
    #TODO
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?' 
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'


#TODO
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> str:
    #TODO
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?' 
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your url for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'



class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

    def _create_agent(self, tools, system_prompt):
        #TODO
        return AgentExecutor

    def run(self, user_input: str):
        #TODO
        return answer


In [24]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_wheather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)

Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода
Вот Ваша ссылка для бронирования отеля: [https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2](https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2) в Санкт-Петербурге с 15 по 24 ноября 2024 года.
Вот Ваша ссылка для заказа билетов: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-24&toId=c2&when=2024-11-15 от Москвы до Санкт-Петербурга на 15.11.2024 / 24.11.2024.
Погода в Санкт-Петербурге сейчас облачная с температурой 8.92°C.
